<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_6_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    684      0 --:--:-- --:--:-- --:--:--   684
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.853125,4.202556,0.216231,03:41


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.983170,3.612297,0.366310,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.016389,3.026686,0.474599,01:24


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.456969,2.577721,0.552139,03:42


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.203107,2.280730,0.590909,03:34
1,1.883727,2.117359,0.635027,03:13


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.741627,1.946455,0.649733,03:15
1,1.510685,1.804912,0.677808,03:23


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.404254,1.681531,0.691176,03:14
1,1.293366,1.621216,0.703209,03:16


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.216086,1.565487,0.707219,03:32
1,1.127105,1.503805,0.716578,03:27


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.098327,1.451626,0.728610,03:36
1,0.941185,1.431964,0.735294,03:37


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.911422,1.405020,0.739305,03:18
1,0.943944,1.379457,0.747326,03:23
2,0.838230,1.290713,0.752674,03:43
3,0.757229,1.284373,0.756684,03:34


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.750963,1.286373,0.770053,03:41
1,0.799373,1.278964,0.767380,03:42
2,0.707870,1.255483,0.771390,03:23
3,0.641414,1.227148,0.768717,03:30


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.646847,1.274784,0.783422,03:18
1,0.718894,1.278034,0.774064,03:37
2,0.639325,1.287986,0.787433,03:13
3,0.583106,1.221181,0.784759,03:37


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.581683,1.285356,0.787433,03:14
1,0.642968,1.253205,0.779412,03:31
2,0.569730,1.226669,0.800802,03:40
3,0.565865,1.213235,0.795455,03:17


In [0]:
# save the best model

learn.save_encoder('ask-6-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [40]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 5, 8, 928, 8, 8, 8, 8, 8, 8, 8, 102, 227, 27, 31, 197, 704, 37, 37, 37, 301, 37, 704, 37, 37, 37, 39, 41, 41, 515, 73, 49, 49, 49, 49, 49, 49, 49, 49, 49, 51, 53, 53, 53, 53, 53, 928, 56, 58, 58, 577, 5, 113, 164, 71, 71, 289, 76, 864, 79, 289, 318, 318, 318, 265, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 260, 289, 289, 289, 265, 80, 80, 80, 80, 80, 80, 274, 662, 1032, 89, 928, 91, 91, 91, 91, 91, 94, 453, 453, 453, 453, 453, 453, 453, 453, 94, 96, 761, 96, 100, 100, 100, 100, 100, 100, 337, 684, 660, 660, 660, 660, 175, 462, 111, 111, 111, 112, 1035, 114, 784, 301, 117, 652, 1032, 655, 753, 438, 125, 194, 125, 125, 125, 125, 125, 125, 778, 82, 741, 319, 113, 89, 763, 257, 151, 807, 194, 157, 157, 163, 165, 171, 172, 289, 289, 289, 172, 877, 71, 1035, 182, 182, 182, 183, 809, 809, 809, 184, 194, 187, 187, 187, 189, 190, 190, 350, 197, 197, 202, 205, 210, 210, 211, 190, 214, 214, 214

In [41]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

579
0.6685912240184757
